在theano中，用符号变量构成表达式，用函数编译表达式，因此不能像在numpy中那样直接使用伪随机数，即便如此，使用伪随机数也并不复杂。

In [1]:
from theano.tensor.shared_randomstreams import RandomStreams
from theano import function

In [2]:
srng = RandomStreams(seed=234)
# 2*2随机矩阵，元素随机抽取自一个均匀分布
rv_u = srng.uniform((2,2))
# 2*2随机矩阵，元素随机抽取自一个正太分布
rv_n = srng.normal((2, 2))
f = function([], rv_u)
g = function([], rv_n, no_default_updates=True) # 不更新rv_n.rng状态
nearly_zeros = function([], rv_u + rv_u -2 * rv_u)

In [3]:
f_val0 = f()
print(f_val0)
f_val1 = f()
print(f_val1)

[[ 0.12672381  0.97091597]
 [ 0.13989098  0.88754825]]
[[ 0.31971415  0.47584377]
 [ 0.24129163  0.42046081]]


In [4]:
g_val0 = g()
print(g_val0)
g_val1 = g()
print(g_val1)

[[ 0.37328447 -0.65746672]
 [-0.36302373 -0.97484625]]
[[ 0.37328447 -0.65746672]
 [-0.36302373 -0.97484625]]


由于g中设置了no_default_updates=True，因此调用g后，随机数生成器的状态不变，所以无论调用g多少次都返回相同的数值。

In [5]:
print(nearly_zeros())

[[ 0.  0.]
 [ 0.  0.]]


 需要特别注意的是，在一次单一的function执行过程中，任何随机变量的抽取操作只执行一次，因此，nearly_zeros函数的返回值为0，即使输出表达式rv_u + rv_u -2 * rv_u中rv_u出现了3次。

## 设置种子

In [6]:
rng_val = rv_u.rng.get_value(borrow=True)
rng_val.seed(89234)
rv_u.rng.set_value(rng_val, borrow=True)

In [7]:
srng.seed(902340)

## 函数之间共享流

与共享变量一样，应用于随机变量的随机数生成器也可以在function之间共享。因此，上边的nearly_zeros函数调用时，也会更新f函数的生成器状态

In [8]:
state_after_v0 = rv_u.rng.get_value().get_state()
nearly_zeros()       # this affects rv_u's generator
v1 = f()
rng = rv_u.rng.get_value(borrow=True)
rng.set_state(state_after_v0)
rv_u.rng.set_value(rng, borrow=True)
v2 = f()             # v2 != v1
v3 = f()             # v3 == v1

In [9]:
v2 == v1

array([[False, False],
       [False, False]], dtype=bool)

In [10]:
v3 == v1

array([[ True,  True],
       [ True,  True]], dtype=bool)

## 在theano图之间拷贝随机变量状态

In [11]:
import theano
import numpy
import theano.tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams
from theano.tensor.shared_randomstreams import RandomStreams

In [12]:
class Graph():
    def __init__(self, seed=123):
        self.rng = RandomStreams(seed)
        self.y = self.rng.uniform(size=(1,))

In [13]:
g1 = Graph(seed=123)
f1 = theano.function([], g1.y)

In [14]:
g2 = Graph(seed=987)
f2 = theano.function([], g2.y)

In [15]:
# 默认情况下，两个函数是不同步的
f1()

array([ 0.72803009])

In [16]:
f2()

array([ 0.55056769])

In [17]:
def copy_random_state(g1, g2):
    if isinstance(g1.rng, MRG_RandomStreams):
        g2.rng.rstate = g1.rng.rstate
    for (su1, su2) in zip(g1.rng.state_updates, g2.rng.state_updates):
        su2[0].set_value(su1[0].get_value())

In [18]:
# 拷贝theano随机数生成器的状态
copy_random_state(g1, g2)

In [19]:
f1()

array([ 0.59044123])

In [20]:
f2()

array([ 0.59044123])